In [1]:
from __future__ import annotations

import os 

from torch.utils.data import random_split

from mmpfn.datasets.pad_ufes_20 import PADUFES20Dataset

import os 
import torch 
import numpy as np 
import pandas as pd

from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, root_mean_squared_error

from mmpfn.models.tabpfn_v2 import TabPFNClassifier
from mmpfn.models.dino_v2.models.vision_transformer import vit_base
from mmpfn.models.tabpfn_v2.constants import ModelInterfaceConfig
from mmpfn.models.tabpfn_v2.preprocessing import PreprocessorConfig
from mmpfn.scripts_finetune.finetune_tabpfn_main import fine_tune_tabpfn

/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
/home/wall/anaconda3/envs/mmpfn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# data_path = os.path.join(os.getenv('HOME'), "workspace/works/tabular_image/MultiModalPFN/mmpfn/data/pad_ufes_20")
data_path = os.path.join(os.getenv('HOME'), "works/research/MultiModalPFN/mmpfn/data/pad_ufes_20")
dataset = PADUFES20Dataset(data_path)

In [3]:
accuracy_scores = []
for seed in range(5):
    torch.manual_seed(seed)
    # np.random.seed(seed)
    # print(f"Finetuning with seed: {seed}")
    
    train_len = int(len(dataset) * 0.8)
    test_len = len(dataset) - train_len

    train_dataset, test_dataset = random_split(dataset, [train_len, test_len])

    X_train = train_dataset.dataset.x[train_dataset.indices]
    y_train = train_dataset.dataset.y[train_dataset.indices]
    X_test = test_dataset.dataset.x[test_dataset.indices]
    y_test = test_dataset.dataset.y[test_dataset.indices]

    for i in range(X_train.shape[1]):
        col = X_train[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1
    for i in range(X_test.shape[1]):
        col = X_test[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1

    torch.cuda.empty_cache()

    save_path_to_fine_tuned_model = "./finetuned_tabpfn_pad_ufes_20.ckpt"
    
    fine_tune_tabpfn(
        # path_to_base_model="auto",
        save_path_to_fine_tuned_model=save_path_to_fine_tuned_model,
        # Finetuning HPs
        time_limit=60,
        finetuning_config={"learning_rate": 0.00001, "batch_size": 1, "max_steps": 100},
        validation_metric="log_loss",
        # Input Data
        X_train=pd.DataFrame(X_train),
        y_train=pd.Series(y_train),
        categorical_features_index=None,
        device="cuda",  # use "cpu" if you don't have a GPU
        task_type="multiclass",
        # Optional
        show_training_curve=False,  # Shows a final report after finetuning.
        logger_level=0,  # Shows all logs, higher values shows less
    )

    # disables preprocessing at inference time to match fine-tuning
    no_preprocessing_inference_config = ModelInterfaceConfig(
        FINGERPRINT_FEATURE=False,
        PREPROCESS_TRANSFORMS=[PreprocessorConfig(name='none')]
    )

    # Evaluate on Test Data
    model_finetuned = TabPFNClassifier(
        model_path=save_path_to_fine_tuned_model,
        inference_config=no_preprocessing_inference_config,
        ignore_pretraining_limits=True,
    )

    clf_finetuned = model_finetuned.fit(X_train, y_train)
    acc_score = accuracy_score(y_test, clf_finetuned.predict(X_test))
    print("accuracy_score (Finetuned):", acc_score)
    accuracy_scores.append(acc_score)

Fine-tuning Steps:   2%|▏         | 2/100 [00:00<01:00,  1.63it/s, Best Val. Loss=0.563, Best Val. Score=-0.563, Training Loss=0.616, Val. Loss=0.563, Patience=49, Utilization=0, Grad Norm=5.16][2025-09-17 23:40:11,752] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:   7%|▋         | 7/100 [00:02<00:41,  2.23it/s, Best Val. Loss=0.551, Best Val. Score=-0.551, Training Loss=0.458, Val. Loss=0.556, Patience=45, Utilization=0, Grad Norm=3.87]


KeyboardInterrupt: 

In [ ]:
# get mean and std of accuracy scores
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)
print("Mean Accuracy:", mean_accuracy)
print("Std Accuracy:", std_accuracy)

Mean Accuracy: 0.8304347826086957
Std Accuracy: 0.01816225962974384
